<a href="https://colab.research.google.com/github/t-nakatani/signate_stu22/blob/exp/signate_stu_v17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

REVISITING FEW-SAMPLE BERT FINE-TUNING - OpenReview にしたがって手始めにpoolerを初期化

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from google.colab import drive
drive.mount('/content/drive')
ROOT = "/content/drive/MyDrive/Colab Notebooks/signate/"

#学習用データと評価用データの読み込み
train = pd.read_csv(os.path.join(ROOT, "train_cleaned.csv"))
test = pd.read_csv(os.path.join(ROOT, "test_cleaned.csv"))

jobdic = dict(zip([1, 2, 3, 4], ['DataScientist', 'ML Engineer', 'Software Engineer', 'Consultant']))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q transformers==3

In [3]:
import numpy as np
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
# from transformers import RobertaTokenizer, RobertaModel
# from transformers import BertModel, AutoTokenizer 

from transformers import AutoTokenizer, AutoModel, AutoConfig
# from transformers import BertTokenizer, BertModel
from torch import optim
from torch import cuda
import time
from matplotlib import pyplot as plt
from tqdm import tqdm

In [4]:
# seeds
SEED = 2022
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

seed_everything(SEED)

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    current_device = torch.cuda.current_device()
    print("Device:", torch.cuda.get_device_name(current_device))

Device: Tesla P100-PCIE-16GB


In [5]:
# Datasetの定義
class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer, max_len):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):  # len(Dataset)で返す値を指定
    return len(self.X)

  def __getitem__(self, index):  # Dataset[index]で返す値を指定
    text = self.X[index]
    inputs = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      pad_to_max_length=True,
      truncation=True
    )
    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    labels = self.y[index]

    return {
      'ids': torch.LongTensor(ids),
      'mask': torch.LongTensor(mask),
      'labels': torch.Tensor(labels)
    }

1. from_pretrainedの文字列でconfig.from_pretrainedを呼び出し
2. config.from_pretrainedの結果をconfigにセット

In [12]:
from pandas._config import config
# BERT分類モデルの定義
class BERTClass(torch.nn.Module):
  def __init__(self, drop_rate, otuput_size):
    super().__init__()
    # self.config = AutoConfig.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=4)
    self.config = AutoConfig.from_pretrained("bert-base-uncased")
    self.bert = AutoModel.from_config(config=self.config)
    self.drop = torch.nn.Dropout(drop_rate)
    self.fc = torch.nn.Linear(768, otuput_size)  # BERTの出力に合わせて768次元を指定
    torch.nn.init.normal_(self.fc.weight, std=0.02)
    torch.nn.init.zeros_(self.fc.bias)

  def forward(self, ids, mask):
    _, out = self.bert(ids, attention_mask=mask)
    out = self.fc(self.drop(out))
    # out = self.bert(ids, attention_mask=mask)
    # out = self.fc(self.drop(out[:, 0, :]))
    return out

In [13]:
def calculate_loss_and_accuracy(model, criterion, loader, device, epoch, max_epoch):
  """ 損失・正解率を計算"""
  model.eval()
  loss = 0.0
  total = 0
  correct = 0
  with torch.no_grad():
    with tqdm(loader) as pbar:
      pbar.set_description('valid')
      for i, data in enumerate(pbar):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        labels = data['labels'].to(device)


        outputs = model(ids, mask)
        loss += criterion(outputs, labels).item()

        pred = torch.argmax(outputs, dim=-1).cpu().numpy() # バッチサイズの長さの予測ラベル配列
        labels = torch.argmax(labels, dim=-1).cpu().numpy()  # バッチサイズの長さの正解ラベル配列
        total += len(labels)
        correct += (pred == labels).sum().item()
        pbar.set_postfix(Loss=loss/(i+1), Accuracy=correct/total)

  return loss / len(loader), correct / total

In [14]:
def train_val(train_loader, val_loader, model_path):
  device = 'cuda' if cuda.is_available() else 'cpu'
  model = BERTClass(DROP_RATE, OUTPUT_SIZE).to(device)
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)
  
  log_valid = []
  best_acc = 0
  

  # train
  for epoch in range(NUM_EPOCHS):
    print('\nepoch:', epoch+1)
    loss_train = correct = total = 0
    model.train()
    with tqdm(train_loader) as pbar:
      pbar.set_description('train')
      for i, data in enumerate(pbar):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        labels = data['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(ids, mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        loss_train += loss.item()
        pred = torch.argmax(outputs, dim=-1).cpu().numpy()
        labels = torch.argmax(labels, dim=-1).cpu().numpy()
        total += len(labels)
        correct += (pred == labels).sum().item()
        pbar.set_postfix(Loss=loss_train/(i+1), Accuracy=correct/total)

    loss_valid, acc_valid = calculate_loss_and_accuracy(model, criterion, val_loader, device, epoch, NUM_EPOCHS)
    log_valid.append([loss_valid, acc_valid])
    if best_acc < acc_valid:
      best_acc = acc_valid
      
      torch.save(model, os.path.join(ROOT, model_path))
      print('==== model saved ====')

  return log_valid

In [15]:
DROP_RATE = 0.1
OUTPUT_SIZE = 4
BATCH_SIZE = 16
NUM_EPOCHS = 10
LEARNING_RATE = 2e-5
N_CV = 5
VERSION = 'v17'

In [16]:
import sklearn.metrics
def train_val_predict(
        df_train,       # 学習用のデータ
        text_column,    # 対象のカラム名
        target_column,  # 目的変数のカラム名
        df_valid=None,  # 検証用データ
        df_val_test=None,       # 予測用データ
        model_file_prefix="",  # 保存時のファイル名識別子
        epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
    ):
  
    # model
    model_path = "{}_{}.pth".format(model_file_prefix, VERSION)
    model = BERTClass(DROP_RATE, OUTPUT_SIZE)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)
    device = 'cuda' if cuda.is_available() else 'cpu'

    # dataset
    df_val_test[target_column].fillna(0)

    max_len = 512
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    dataset_train = CreateDataset(df_train[text_column].values, pd.get_dummies(df_train[target_column]).values, tokenizer, max_len)
    dataset_val = CreateDataset(df_valid[text_column].values, pd.get_dummies(df_valid[target_column]).values, tokenizer, max_len)
    dataset_test = CreateDataset(df_val_test[text_column].values, pd.get_dummies(df_val_test[target_column]).values, tokenizer, max_len)

    dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
    dataloader_valid = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)
    dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)

    # train
    print('\n================  start train  ================')
    metric = train_val(dataloader_train, dataloader_valid, model_path)
    print('================  end train  ================\n')

    # predict
    model = torch.load(os.path.join(ROOT, model_path))
    model.eval()
    pred_y_list = []
    emb_list = []
    with torch.no_grad():
      with tqdm(dataloader_test) as pbar:
        pbar.set_description('test')
        for data in pbar:
          ids = data['ids'].to(device)
          mask = data['mask'].to(device)
          labels = data['labels'].to(device)

          output = model.forward(ids, mask)
          pred = torch.argmax(output, dim=-1).cpu().numpy()

          pred_y_list.extend(pred)
          emb_list.extend(output.cpu().numpy())
    return metric, pred_y_list, emb_list

In [17]:
import sklearn.model_selection

text_column, target_column, n_splits = ('description', 'jobflag', N_CV)

df = pd.concat([train, test], ignore_index=True, sort=False)
df_train = df[df[target_column].notnull()]
df_test = df[df[target_column].isnull()]

df_train_idx = df_train.index

# store result
df_pred = pd.DataFrame(df.index, columns=["index"]).set_index("index")
df_emb = pd.DataFrame(df.index, columns=["index"]).set_index("index")
df_emb_pred = None
metric_list = []
all_emb = []

# cross validation
kf = sklearn.model_selection.StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1234)
for i, (train_idx, test_idx) in enumerate(kf.split(df_train, df_train[target_column])):
  df_train_sub = df_train.iloc[train_idx]
  df_test_sub = df_train.iloc[test_idx]

  df_val_test = pd.concat([df_test_sub, df_test], ignore_index=True, sort=False)

  model_file_prefix = "cv_{}".format(i)

  # train
  metric, pred_y_list, emb_list = train_val_predict(
      df_train=df_train_sub, 
      text_column=text_column,
      target_column=target_column, 
      df_valid=df_test_sub,
      df_val_test=df_val_test,
      model_file_prefix=model_file_prefix,
  )
  metric_list.append(metric)
  all_emb.append(emb_list)
  # break

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]


================  start train  ================

epoch: 1


valid: 100%|██████████| 19/19 [00:05<00:00,  3.37it/s, Accuracy=0.309, Loss=1.29]


==== model saved ====

epoch: 2


valid: 100%|██████████| 19/19 [00:05<00:00,  3.38it/s, Accuracy=0.434, Loss=1.2]


...

valid: 100%|██████████| 19/19 [00:05<00:00,  3.36it/s, Accuracy=0.743, Loss=0.976]


==== model saved ====

epoch: 10


valid: 100%|██████████| 19/19 [00:05<00:00,  3.36it/s, Accuracy=0.759, Loss=0.848]


==== model saved ====
================  end train  ================



test: 100%|██████████| 114/114 [00:33<00:00,  3.40it/s]


In [18]:
all_emb_ = np.array([np.array(emb)[-len(df_test):] for emb in all_emb])

In [19]:
mean_df = pd.DataFrame([[np.mean(arr_val) for arr_val in arr_vec] for arr_vec in np.moveaxis(all_emb_, 0, 2)])
preds = [np.argmax(arr)+1 for arr in mean_df.to_numpy()]
df_submit = pd.DataFrame([df_test.id.to_list(), preds]).T
df_submit.to_csv(os.path.join(ROOT, f'{VERSION}_esemble.csv'), index=None, header=None)

In [20]:
acc_list = []
for item in metric_list:
  acc_list.append((np.max([acc for loss, acc in item])))
print(np.mean(acc_list))

0.7400881535521974
